In [3]:
K8S_PROXY_ADDR='127.0.0.1:8001'
K8S_NAMESPACE='mdt'
APP_NAME='sklearn-classification'
MODEL_NAME='clipper-sklearn-predict'
MODEL_FILE='classification.pkl'
REPO_URL='658391232643.dkr.ecr.us-west-2.amazonaws.com'
VERSION = 1

Train the Model and persist it into a pickle file

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.externals import joblib

data = load_iris()

X, y = data["data"], data["target"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
                       oob_score=False, random_state=0, verbose=0, warm_start=False)

print(clf.feature_importances_)

print(classification_report(y_test, clf.predict(
    X_test), target_names=data["target_names"]))

joblib.dump(clf, 'classification.pkl')


[0.13991367 0.01410133 0.33966798 0.50631702]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       0.94      1.00      0.97        15
   virginica       1.00      0.94      0.97        16

   micro avg       0.98      0.98      0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50



/Users/gtao/Library/Python/2.7/lib/python/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


['classification.pkl']

Connect to clipper and register App

In [6]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
from clipper_admin.deployers import python as python_deployer
manager = KubernetesContainerManager(
    kubernetes_proxy_addr=K8S_PROXY_ADDR, namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()

clipper_conn.register_application(
   name = APP_NAME, input_type = 'doubles', default_output = '0', slo_micros = 100000000)

18-10-02:10:43:08 INFO     [kubernetes_container_manager.py:317] [default-cluster] Found 6 nodes: ec2-54-68-64-106.us-west-2.compute.amazonaws.com, ec2-34-219-191-153.us-west-2.compute.amazonaws.com, ec2-18-236-197-218.us-west-2.compute.amazonaws.com, ec2-54-212-98-152.us-west-2.compute.amazonaws.com, ec2-18-236-106-32.us-west-2.compute.amazonaws.com, ec2-52-13-37-198.us-west-2.compute.amazonaws.com
18-10-02:10:43:08 INFO     [kubernetes_container_manager.py:328] [default-cluster] Setting Clipper mgmt port to 30091
18-10-02:10:43:08 INFO     [kubernetes_container_manager.py:338] [default-cluster] Setting Clipper query port to 32273
18-10-02:10:43:08 INFO     [kubernetes_container_manager.py:361] [default-cluster] Setting Clipper metric port to 30164
18-10-02:10:43:08 INFO     [clipper_admin.py:151] [default-cluster] Successfully connected to Clipper cluster at 127.0.0.1:8001/api/v1/namespaces/mdt/services/query-frontend-at-default-cluster:1337/proxy
18-10-02:10:43:08 INFO     [clipper_

Create a prediction function and deploy it to clipper

In [9]:
from sklearn.externals import joblib
clf = joblib.load(MODEL_FILE)

def predict_wrapper(X):
    print("inputs {}".format(X))
    try:
        result = clf.predict(X)
        print("result is {}".format(result))
        ret = [str(i) for i in result]
        print("return is {}".format(ret))
        return ret
    except Exception as e:
        print(e)
        return [str(e)]

python_deployer.deploy_python_closure(clipper_conn,
                                      name=MODEL_NAME,
                                      version=VERSION,
                                      input_type="doubles",
                                      func=predict_wrapper,
                                      registry=REPO_URL,
                                      pkgs_to_install=['sklearn'])

18-10-02:10:45:05 INFO     [deployer_utils.py:41] Saving function to /var/folders/nk/xbb077ys5nd_t4mpb2s4fwfc0000gn/T/tmprdclQ1clipper
18-10-02:10:45:05 INFO     [deployer_utils.py:51] Serialized and supplied predict function
18-10-02:10:45:05 INFO     [python.py:192] Python closure saved
18-10-02:10:45:05 INFO     [python.py:198] Using Python 2 base image
18-10-02:10:45:05 INFO     [clipper_admin.py:467] [default-cluster] Building model Docker image with model data from /var/folders/nk/xbb077ys5nd_t4mpb2s4fwfc0000gn/T/tmprdclQ1clipper
18-10-02:10:45:05 INFO     [clipper_admin.py:472] [default-cluster] Step 1/3 : FROM clipper/python-closure-container:develop
18-10-02:10:45:05 INFO     [clipper_admin.py:472] [default-cluster]  ---> 76a098e0f6ee
18-10-02:10:45:05 INFO     [clipper_admin.py:472] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install sklearn
18-10-02:10:45:05 INFO     [clipper_admin.py:472] [default-cluster]  ---> Using cache
18-10-02:10:45:05 I

Link the model to application

In [11]:
clipper_conn.link_model_to_app(app_name=APP_NAME, model_name=MODEL_NAME)

18-10-02:10:45:35 INFO     [clipper_admin.py:277] [default-cluster] Model clipper-sklearn-predict is now linked to application sklearn-classification


In [22]:
import requests, json, numpy as np
def predict():
    headers = {"Content-type": "application/json"}
    data=json.dumps({"input": [5.9, 3.0, 5.1, 1.8]})
    print(data)
    url = "http://{}/api/v1/namespaces/{}/services/query-frontend-at-default-cluster:1337/proxy/{}/predict".format(K8S_PROXY_ADDR,K8S_NAMESPACE,APP_NAME)
    res = requests.post(url, headers=headers, data=data)
    return res.json()

print(predict())

{"input": [5.9, 3.0, 5.1, 1.8]}
{u'default': False, u'output': 2, u'query_id': 37}
